In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_csv("preprocessed_df2.csv")

In [5]:
data = data.drop(columns = ['FlightDate', 'Tail_Number', 'Flight_Number_Reporting_Airline', 'OriginAirportSeqID', 'OriginCityName', 'OriginState', 'DestAirportSeqID', 'DestCityName', 'DestState', 'MinSince'])


In [7]:
data = pd.get_dummies(data)

In [9]:
data['Delay'] = data['DepDelay'].apply(lambda x: 1 if x < 0 else 0)


In [10]:
data = data.drop(columns = ['DepDelay'])

In [11]:
from sklearn.model_selection import train_test_split

X = data.drop('Delay', axis=1)
y = data['Delay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=1)


In [12]:
#X_train = X_train.iloc[:10000]
#y_train = y_train.iloc[:10000]
#X_test = X_test.iloc[:10000]
#y_test = y_test.iloc[:10000]


In [13]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

xgb_classifier = xgb.XGBClassifier()

param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 0.1, 0.2]
}

grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_xgb_classifier = grid_search.best_estimator_

y_pred = best_xgb_classifier.predict(X_test)

testing_accuracy = accuracy_score(y_test, y_pred)

print(testing_accuracy)

KeyboardInterrupt: 

In [ ]:
from sklearn.svm import SVC

svm_classifier = SVC()

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_svm_classifier = grid_search.best_estimator_

y_pred = best_svm_classifier.predict(X_test)

testing_accuracy = accuracy_score(y_test, y_pred)

print(testing_accuracy)
